In [ ]:
import os
import json
import time
import hashlib
import requests
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from PIL import Image
from io import BytesIO
import base64
import threading

# === Config ===
CONTROLLER = "http://127.0.0.1:9000/generate"
CLIENT_ID = "my-client-1"
SAVE_DIR = "outputs"
PROMPTS = ["a cat", "a dog", "a castle", "a dragon"]*4  # Add more as needed

# === Ensure save dir with date prefix ===
date_prefix = datetime.now().strftime("%Y-%m-%d")
output_path = os.path.join(SAVE_DIR, date_prefix)
os.makedirs(output_path, exist_ok=True)

# === Determine starting index ===
def get_next_index(path):
    existing = [
        f for f in os.listdir(path)
        if f.endswith(".json") and f[:4].isdigit()
    ]
    if not existing:
        return 0
    max_index = max(int(f[:4]) for f in existing)
    return max_index + 1

index_lock = threading.Lock()
index_counter = get_next_index(output_path)

# === Save function ===
def save_image_and_metadata(img_b64, metadata):
    global index_counter
    with index_lock:
        index = index_counter
        index_counter += 1
    prefix = f"{index:04d}"
    
    # Compute hash to avoid naming collisions
    meta_str = json.dumps(metadata, sort_keys=True)
    digest = hashlib.sha1(meta_str.encode()).hexdigest()[:8]
    
    img_filename = f"{prefix}_{digest}.png"
    json_filename = f"{prefix}_{digest}.json"

    # Decode and save image
    image = Image.open(BytesIO(base64.b64decode(img_b64.split(',')[-1])))
    image.save(os.path.join(output_path, img_filename))

    # Save metadata
    metadata["client_timestamp"] = datetime.utcnow().isoformat()
    with open(os.path.join(output_path, json_filename), "w") as f:
        json.dump(metadata, f, indent=2)

    print(f"Saved: {img_filename}, {json_filename}")

# === Request function ===
def send(prompt, width, height):
    try:
        resp = requests.post(
            CONTROLLER,
            json={"prompt": prompt, "width": width, "height": height},
            headers={"X-Client-ID": CLIENT_ID},
            timeout=300
        )
        resp.raise_for_status()
        data = resp.json()
        if "image" in data and "metadata" in data:
            save_image_and_metadata(data["image"], data["metadata"])
        else:
            print(f"Failed: Missing image or metadata for prompt: {prompt}")
    except Exception as e:
        print(f"Error for prompt '{prompt}': {e}")

# === Main Execution ===
if __name__ == "__main__":
    start = time.time()
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(send, p, 1024, 1024) for p in PROMPTS]
        [f.result() for f in futures]
    print("Done in", round(time.time() - start, 2), "seconds.")
